In [0]:
%run ../config/init_config


In [0]:
from pyspark.sql.functions import from_json, to_date, lit, col, row_number, broadcast, count, sum
from pyspark.sql.window import Window

In [0]:
def process_order_customer():
    current_customer_silver = spark.table("current_customer_silver")

    query = (spark.readStream
                .table("order_silver")
                .withColumnRenamed("customer_id", "customer_id_left")
                .join(current_customer_silver, col("customer_id_left") == col("customer_id"), "inner")
                .drop("customer_id_left")
                .drop("expired_at")
                .drop("is_current")
                .writeStream
                .format("delta")
                .option("mergeSchema", "true")
                .option("checkpointLocation", silver_checkpoint_path + "/order_customer")
                .trigger(availableNow=True)
                .table("order_customer")
                )
                        

    query.awaitTermination()

In [0]:
process_order_customer()